In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 1. Convert image to grayscale to simplify computation

image = cv2.imread("Downloads/test_image.jpg")
lane_image = np.copy(image)

# 2. apply canny edge detection to find steep gradient change areas.
def canny(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # Take out gaussian noise
    blur= cv2.GaussianBlur(gray, (5,5), 0)
    #3. Find edges: we find areas where gradient is steep
    # compute gradient of f(x,y) with Canny function - Canny function finds derivative - rate of change of intensity
    canny = cv2.Canny(blur, 50, 150)
    return canny

# 3. Identify region of interest where lanes are present
def region_of_interest(image):
    height = image.shape[0]
    # generate a polygonal contour.
    polygons = np.array([
        [(200,height), (1100, height), (550, 200)]
    ])
    
    # need a black mask
    mask = np.zeros_like(image) 
    # Now we fill the black mask with our triangle
    cv2.fillPoly(mask, polygons, 255)
    # bit-wise multipln of polygon mask and canny image
    masked_image = cv2.bitwise_and(image, mask)
    
    return masked_image

def display_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for x1, y1, x2, y2 in lines:
            cv2.line(line_image, (x1, y1), (x2, y2), (255,0,0), 10) # draw over a line with b/w coordinates x1,y1 and x2,y2, colour - 255,0,0(blue), thickness - 10
    return line_image
  
# 5. Now we build a best fit line by finding the avg slope of lines   
# a. first finding the avg slope, intercept
def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1,x2), (y1,y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
    # extract the avg slopes and intercepts (m,c)
    left_fit_average = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)
    # also need to specify the coordinates (x,y) to build lines with (m,c)
    left_line = make_coordinates(image, left_fit_average)
    right_line = make_coordinates(image, right_fit_average)
    return np.array([left_line, right_line])
        
def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1, y1, x2, y2])
    
canny_image = canny(lane_image)
cropped_image = region_of_interest(canny_image)  

# 4. pinpont the edges using Hough Transform
# 2nd and 3rd arguments - specify the resolution of the Hough accumulator array (2D array consisting of bins that collect votes)
# each bin represent distinct value of rho, theta
# rho - distance resolution, theta - angle resolution
# larger the bins, less precision
# Finally, how do we know which bins to choose, optimal number of votes to choose a bin??
# Using threshold - bin containing 
    
lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5) 
averaged_lines = average_slope_intercept(lane_image, lines)
line_image = display_lines(lane_image, averaged_lines)
# combine line_image and lane_image
combo_image = cv2.addWeighted(lane_image, 0.8, line_image, 1, 1)


In [3]:
cv2.imshow("result", combo_image)
cv2.waitKey(0)

-1

In [2]:
# Let us implement this for a video

cap = cv2.VideoCapture("Downloads/test2.mp4")
# while capturing has been initialized, we enter a loop
while(cap.isOpened()):
    _, frame = cap.read() # cap.read() decodes every video frame, '_' --> boolean placeholder that is not reqd, 'frame' - current frame
    
    canny_image = canny(frame) 
    cropped_image = region_of_interest(canny_image)  
    lines = cv2.HoughLinesP(cropped_image, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5) 
    averaged_lines = average_slope_intercept(frame, lines)
    line_image = display_lines(frame, averaged_lines)
    combo_image = cv2.addWeighted(frame, 0.8, line_image, 1, 1)
    cv2.imshow("result", combo_image)
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


